# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [122]:
import os
import sys

sys.path.append('..')
# Set your preferred name
USER = "Juncheng"
# Set the data and output directories
DATA_ROOT = '/Users/Henry/Desktop/uiuc/大二下/independent study/part 1'
OUTPUT_ROOT = '../pud-output1'
TRAIT = 'Peptic ulcer disease'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [123]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [124]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:]

['TCGA_Colon_and_Rectal_Cancer_(COADREAD)',
 'TCGA_Ocular_melanomas_(UVM)',
 'TCGA_Lower_Grade_Glioma_(LGG)',
 'CrawlData.ipynb',
 'TCGA_Mesothelioma_(MESO)',
 'TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Kidney_Clear_Cell_Carcinoma_(KIRC)',
 'TCGA_Head_and_Neck_Cancer_(HNSC)',
 'TCGA_Lung_Cancer_(LUNG)',
 'TCGA_Ovarian_Cancer_(OV)',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_lower_grade_glioma_and_glioblastoma_(GBMLGG)',
 'TCGA_Liver_Cancer_(LIHC)',
 'TCGA_Uterine_Carcinosarcoma_(UCS)',
 'TCGA_Cervical_Cancer_(CESC)',
 'TCGA_Colon_Cancer_(COAD)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Kidney_Papillary_Cell_Carcinoma_(KIRP)',
 'TCGA_Endometrioid_Cancer_(UCEC)',
 'TCGA_Rectal_Cancer_(READ)',
 'TCGA_Melanoma_(SKCM)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Prostate_Cancer_(PRAD)',
 'TCGA_Lung_Squamous_Cell_Carcinoma_(LUSC)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Large_Bcell_Lymphoma_(DLBC)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM

If no match is found, jump directly to GEO in Part 2.2

In [125]:
trait_subdir = "TCGA_Lung_Cancer_(LUNG)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_LUNG_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.LUNG.sampleMap_HiSeqV2_PANCAN.gz')

In [126]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [127]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [128]:
clinical_data_cols[:10]

['ABSOLUTE_Ploidy',
 'ABSOLUTE_Purity',
 'AKT1',
 'ALK_translocation',
 'BRAF',
 'CBL',
 'CTNNB1',
 'Canonical_mut_in_KRAS_EGFR_ALK',
 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2',
 'EGFR']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [129]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['ABSOLUTE_Ploidy', 'ABSOLUTE_Purity', 'AKT1', 'ALK_translocation', 'BRAF', 'CBL', 'CTNNB1', 'Canonical_mut_in_KRAS_EGFR_ALK', 'Cnncl_mt_n_KRAS_EGFR_ALK_RET_ROS1_BRAF_ERBB2_HRAS_NRAS_AKT1_MAP2', 'EGFR', 'ERBB2', 'ERBB4', 'Estimated_allele_fraction_of_a_clonal_varnt_prsnt_t_1_cpy_pr_cll', 'Expression_Subtype', 'HRAS', 'KRAS', 'MAP2K1', 'MET', 'NRAS', 'PIK3CA', 'PTPN11', 'Pathology', 'Pathology_Updated', 'RET_translocation', 'ROS1_translocation', 'STK11', 'WGS_as_of_20120731_

In [130]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_after_initial_treatment', 'stopped_smoking_year', 'year_of_initial_pathologic_diagnosis', 'year_of_tobacco_smoking_onset']

candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [131]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [70.0, 67.0, 79.0, 68.0, 66.0],
 'days_to_birth': [-25752.0, -24532.0, -29068.0, -24868.0, -24411.0],
 'days_to_collection': [nan, nan, nan, nan, nan],
 'days_to_death': [nan, nan, '121', nan, nan],
 'days_to_initial_pathologic_diagnosis': [0.0, 0.0, 0.0, 0.0, 0.0],
 'days_to_last_followup': ['0', '1523', nan, '607', '426'],
 'days_to_new_tumor_event_after_initial_treatment': [nan,
  nan,
  nan,
  334.0,
  183.0],
 'stopped_smoking_year': [nan, nan, nan, nan, 1987.0],
 'year_of_initial_pathologic_diagnosis': [2009.0,
  2007.0,
  2007.0,
  2009.0,
  2009.0],
 'year_of_tobacco_smoking_onset': [nan, nan, nan, nan, 1963.0]}

In [132]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [133]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['MALE', 'MALE', 'FEMALE', 'MALE', 'MALE']}

In [134]:
gender_col = 'gender'

In [135]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [136]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [137]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Peptic ulcer disease,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-05-4244-01,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,3.634614,0.677218,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.195527,-0.324695,0.193167
TCGA-05-4249-01,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,4.908014,0.888518,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.276027,-1.219395,1.850967
TCGA-05-4250-01,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,0.489514,-0.086682,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.714373,0.112305,-0.532833
TCGA-05-4382-01,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,5.730914,-0.086682,1.013722,-1.587117,0.359267,2.59051,0.806623,0.367373,-0.494595,-1.062233
TCGA-05-4384-01,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,4.163014,0.601218,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,1.469373,0.194605,1.404667


In [138]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 1100 samples.


In [139]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Peptic ulcer disease', the least common label is '0' with 110 occurrences. This represents 10.00% of the dataset.
The distribution of the feature 'Peptic ulcer disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 60.0
  50% (Median): 67.0
  75%: 73.0
Min: 38.0
Max: 90.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 443 occurrences. This represents 40.27% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [140]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [141]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [142]:
dataset = 'GEO'
trait_subdir = "Peptic-ulcer-disease"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE60427', 'GSE43580', 'GSE32174', 'GSE207846']

Repeat the below steps for all the accession numbers

Fixed function name

In [143]:
cohort = accession_num = "GSE43580"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('/Users/Henry/Desktop/uiuc/大二下/independent study/part 1/GEO/Peptic-ulcer-disease/GSE43580/GSE43580_family.soft.gz',
 '/Users/Henry/Desktop/uiuc/大二下/independent study/part 1/GEO/Peptic-ulcer-disease/GSE43580/GSE43580_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [144]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"SBV - Gene Expression Profiles of Lung Cancer Tumors - Adenocarcinomas and Squamous Cell Carcinomas"
!Series_summary	"This dataset encompassing the profiles of 150 lung cancer tumors was developed to serve as test dataset in the SBV IMPROVER Diagnostic Signature Challenge (sbvimprover.com). The aim of this subchallenge was to verify that it is possible to extract a robust diagnostic signature from gene expression data that can identify stages of different types of lung cancer. Participants were asked to develop and submit a classifier that can stratify lung cancer patients in one of four groups – Stage 1 of Adenocarcinoma (AC Stage 1), Stage 2 of Adenocarcinoma (AC Stage 2), Stage 1 of Squamous cell carcinoma (SCC Stage 1) or Stage 2 of Squamous cell carcinoma (SCC Stage 2). The classifier could be built by using any publicly available gene expression data with related histopathological information and was tested on the independent dataset described here."
!Series_overal

In [145]:
clinical_data.head()

,!Sample_geo_accession,GSM1065725,GSM1065726,GSM1065727,GSM1065728,GSM1065729,GSM1065730,GSM1065731,GSM1065732,GSM1065733,...,GSM1065865,GSM1065866,GSM1065867,GSM1065868,GSM1065869,GSM1065870,GSM1065871,GSM1065872,GSM1065873,GSM1065874
0,!Sample_characteristics_ch1,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,gender: male,...,gender: male,gender: male,gender: female,gender: male,gender: male,gender: male,gender: male,gender: female,gender: male,gender: male
1,!Sample_characteristics_ch1,age at excision (years): 65,age at excision (years): 61,age at excision (years): 43,age at excision (years): 44,age at excision (years): 60,age at excision (years): 58,age at excision (years): 67,age at excision (years): 52,age at excision (years): 43,...,age at excision (years): 80,age at excision (years): 57,age at excision (years): 69,age at excision (years): 53,age at excision (years): 53,age at excision (years): 64,age at excision (years): 58,age at excision (years): 64,age at excision (years): 46,age at excision (years): 56
2,!Sample_characteristics_ch1,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,...,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Caucasian,ethnicity: Asian/Pacific Islander,ethnicity: Caucasian,ethnicity: Caucasian
3,!Sample_characteristics_ch1,clinical diagnosis patient: Squamous cell carc...,clinical diagnosis patient: Hypertension (Stat...,clinical diagnosis patient: Squamous cell carc...,clinical diagnosis patient: Simple chronic bro...,clinical diagnosis patient: Squamous cell carc...,clinical diagnosis patient: Squamous cell carc...,clinical diagnosis patient: Squamous cell carc...,clinical diagnosis patient: Squamous cell carc...,clinical diagnosis patient: Squamous cell carc...,...,clinical diagnosis patient: Adenocarcinoma of ...,clinical diagnosis patient: Chronic bronchitis...,clinical diagnosis patient: Adenocarcinoma of ...,clinical diagnosis patient: Adenocarcinoma of ...,clinical diagnosis patient: Adenocarcinoma of ...,clinical diagnosis patient: Chronic bronchitis...,clinical diagnosis patient: Car accident with ...,clinical diagnosis patient: Diabetes (Status O...,clinical diagnosis patient: Squamous cell carc...,clinical diagnosis patient: Squamous cell carc...
4,!Sample_characteristics_ch1,bmi: 33.9100346,bmi: 27.6816609,bmi: 26.25958475,bmi: 25.40170132,bmi: 25.48476454,bmi: 25.69706994,bmi: 29.3877551,bmi: 28.40550436,bmi: 22.85714286,...,bmi: 21.51385851,bmi: 20.2020202,bmi: 28.04037814,bmi: 24.6097337,bmi: 25.71166208,bmi: 25.55932916,bmi: 18.51851852,bmi: 23.01117686,bmi: 31.77393186,bmi: 28.39372634


In [146]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['gender: male', 'gender: female', 'age at excision (years): NA'],
 1: ['age at excision (years): 65',
  'age at excision (years): 61',
  'age at excision (years): 43',
  'age at excision (years): 44',
  'age at excision (years): 60',
  'age at excision (years): 58',
  'age at excision (years): 67',
  'age at excision (years): 52',
  'age at excision (years): 66',
  'age at excision (years): 47',
  'age at excision (years): 56',
  'age at excision (years): 62',
  'age at excision (years): 69',
  'age at excision (years): 49',
  'age at excision (years): 68',
  'age at excision (years): 70',
  'age at excision (years): 46',
  'age at excision (years): 63',
  'age at excision (years): 57',
  'age at excision (years): 39',
  'age at excision (years): 55',
  'age at excision (years): 71',
  'age at excision (years): 54',
  'age at excision (years): 72',
  'age at excision (years): 74',
  'age at excision (years): 59',
  'age at excision (years): 73',
  'age at excision (years): 77',
  

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [147]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.

Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Peptic ulcer disease\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Peptic ulcer disease\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

Seems like the trait_type should not be binary, since there are multiples types, but not sure how to fix this

In [148]:

is_gene_availabe = True
trait_row = 2
age_row = 1
gender_row = 0

trait_type = 'binary'

In [149]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [150]:
def convert_trait(tissue_type):
    """
    Convert tissue type to epilepsy presence (binary).
    Assuming epilepsy presence for 'Hippocampus' tissue.
    """
    if tissue_type =='hp_presence: yes':
        return 1  # Epilepsy present
    else:
        return 0  # Epilepsy not present

def convert_age(age_string):
    """
    Convert age string to a numerical value.
    """
    if age_string.startswith('age: '):
        try:
            return int(age_string.split('age: ')[1].rstrip(''))  # Extracting numeric part
        except ValueError:
            return None  # In case of parsing error
    else:
        return None  # If age string does not follow expected format

def convert_gender(gender_string):
    """
    Convert gender string to a binary representation.
    Female -> 0, Male -> 1, Unknown -> None
    """
    if gender_string == 'gender: Female':
        return 0
    elif gender_string == 'gender: Male':
        return 1
    else:
        return None  # In case of unknown gender

# Example usage
example_tissue = 'hp_presence: yes'
example_age = 'age: 23'
example_gender = 'gender: Male'

converted_trait = convert_trait(example_tissue)
converted_age = convert_age(example_age)
converted_gender = convert_gender(example_gender)

converted_trait, converted_age, converted_gender

(1, 23, 1)

In [151]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM1065725,GSM1065726,GSM1065727,GSM1065728,GSM1065729,GSM1065730,GSM1065731,GSM1065732,GSM1065733,GSM1065734,...,GSM1065865,GSM1065866,GSM1065867,GSM1065868,GSM1065869,GSM1065870,GSM1065871,GSM1065872,GSM1065873,GSM1065874
Peptic ulcer disease,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Age,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
Gender,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


### Genetic data preprocessing and final filtering

In [152]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM1065725,GSM1065726,GSM1065727,GSM1065728,GSM1065729,GSM1065730,GSM1065731,GSM1065732,GSM1065733,GSM1065734,...,GSM1065865,GSM1065866,GSM1065867,GSM1065868,GSM1065869,GSM1065870,GSM1065871,GSM1065872,GSM1065873,GSM1065874
ID,,,,,,,,,,,,,,,,,,,,,
1007_s_at,3436.7820,3304.27300,2197.12400,2815.7060,3707.87400,2321.4000,1794.30100,4989.02400,2451.17400,4437.60000,...,2531.44200,3057.93800,3302.16700,2876.34200,2302.4420,2435.69600,1403.93300,3188.060000,1052.40600,2732.80600
1053_at,895.8463,1178.84400,493.41270,487.9837,807.11980,898.1168,487.91890,956.28450,523.00660,742.22830,...,1091.43300,863.17980,279.82880,1433.30200,483.4991,472.51840,754.12210,325.676800,438.84010,575.72250
117_at,400.9193,486.10700,310.81070,421.0068,911.87270,303.4280,1153.86800,1217.01300,844.26650,276.02070,...,570.08420,1756.28700,307.40450,986.43420,284.8585,297.93990,1233.08500,344.073700,242.35270,505.02750
121_at,259.1138,384.20850,310.86190,299.6615,356.11960,386.1635,407.97170,315.42400,445.42260,299.75130,...,349.71690,367.64690,398.81310,267.62490,288.5833,438.30970,619.76300,372.962400,423.31890,437.28310
1255_g_at,127.8382,32.96813,40.59212,80.6160,58.36799,142.0764,53.78057,40.73191,11.78208,76.67336,...,16.27517,40.96819,45.35161,35.15165,104.7598,26.38769,19.48837,6.667458,27.64435,63.39404


In [153]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['1007_s_at',
 '1053_at',
 '117_at',
 '121_at',
 '1255_g_at',
 '1294_at',
 '1316_at',
 '1320_at',
 '1405_i_at',
 '1431_at',
 '1438_at',
 '1487_at',
 '1494_f_at',
 '1552256_a_at',
 '1552257_a_at',
 '1552258_at',
 '1552261_at',
 '1552263_at',
 '1552264_a_at',
 '1552266_at']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [154]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at', '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at', '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at', '1552263_at', '1552264_a_at', '1552266_at']\n"

If not required, jump directly to the gene normalization step

In [155]:
requires_gene_mapping = True

In [156]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens

Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [157]:
if requires_gene_mapping:
    print(f'''
    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {gene_annotation_summary}
    ''')


    As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
    probe_name_key = 'key_name1'
    gene_name_key = 'key_name2'

    Gene annotation dictionary:
    {'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Da

In [158]:
#genetic_data.head()
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'SYMBOL'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)
#genetic_data.head()

KeyError: "['SYMBOL'] not in index"

In [ ]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

In [ ]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [ ]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 48 samples.


In [ ]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Peptic ulcer disease', the least common label is '0.0' with 9 occurrences. This represents 18.75% of the dataset.
The distribution of the feature 'Peptic ulcer disease' in this dataset is fine.

Quartiles for 'Age':
  25%: 38.0
  50% (Median): 43.0
  75%: 56.25
Min: 19.0
Max: 80.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 23 occurrences. This represents 47.92% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [ ]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data, note='')

In [ ]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [ ]:
from utils import *

In [ ]:
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH, 'has_age')
best_cohort

'Xena'

In [ ]:
ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,1100,
1,GSE32174,True,True,False,True,True,48,


In [ ]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, best_cohort+'.csv'))

In [ ]:
merged_data.head()

,Peptic ulcer disease,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,70.0,1.0,-0.056592,2.128674,-0.531035,-0.185972,-0.190878,-0.11891,-0.659794,...,3.634614,0.677218,0.015022,-1.155117,0.528367,2.75061,0.428423,-0.195527,-0.324695,0.193167
1,1,67.0,1.0,-0.569692,-0.318126,-0.531035,0.237428,0.011822,-0.00611,1.018406,...,4.908014,0.888518,-0.031778,-0.393117,1.632467,2.33551,-0.145577,-0.276027,-1.219395,1.850967
2,1,79.0,0.0,-0.899992,-0.918126,-0.531035,-0.621472,-0.454178,0.58719,-0.198894,...,0.489514,-0.086682,-0.748878,-1.173917,1.966367,-1.28139,-0.374877,0.714373,0.112305,-0.532833
3,1,68.0,1.0,0.667508,3.895974,-0.221535,-0.367272,0.131822,0.97629,-0.494994,...,5.730914,-0.086682,1.013722,-1.587117,0.359267,2.59051,0.806623,0.367373,-0.494595,-1.062233
4,1,66.0,1.0,-0.316892,2.380374,0.156865,0.232828,0.054822,0.27059,0.128606,...,4.163014,0.601218,-0.060978,-0.435417,1.111467,-1.28139,-0.128477,1.469373,0.194605,1.404667


In [ ]:
# If both age and gender features are available, select 'age' as the condition.
condition = 'Age'
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

In [ ]:
X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values
print(X.shape)
print(Y)

(1100, 17339)
[1. 1. 1. ... 1. 1. 1.]


Select the appropriate regression model depending on whether the dataset shows batch effect.

In [ ]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [ ]:

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [ ]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 56.36% ± 3.30%


In [ ]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)
/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)
/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)
/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)
/Users/Henry/Library/Python/3.7/lib/python/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in true_divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 89.27% ± 3.21%


In [ ]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/home/yxcheng/anaconda3/envs/ai4sci/lib/python3.10/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 5. Discussion and report

In [ ]:

feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0077
Found 10 genes with non-zero coefficients associated with the trait 'Lung-Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
   Variable  Coefficient  Absolute Coefficient
    SNORA81     0.605617              0.605617
    SNORD89    -0.561992              0.561992
     OR4F17     0.468452              0.468452
SNORD115-26    -0.289358              0.289358
     OR2L1P    -0.278115              0.278115
    TBC1D3C     0.248994              0.248994
     DMRTA2    -0.194621              0.194621
     IFIT1B    -0.188779              0.188779
    PPIAL4E     0.150659              0.150659
     TTTY16    -0.082366              0.082366


/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [ ]:

interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: -0.0077
Found 14 significant genes associated with the trait 'Lung-Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
Variable  Coefficient      p_value  corrected_p_value
   MEX3A     0.061062 2.147189e-07           0.003723
 SLC19A1    -0.038907 6.271788e-07           0.005437
  CRABP2     0.036440 1.633672e-06           0.009442
   HMGB3     0.042606 8.393633e-06           0.020791
 DEFB118     0.028296 7.138943e-06           0.020791
     IL3    -0.016966 5.077872e-06           0.020791
  FAM83A     0.043838 8.074414e-06           0.020791
    RCC1     0.041270 1.786386e-05           0.028158
    NUF2     0.068410 1.754965e-05           0.028158
    SPP1     0.032014 1.782137e-05           0.028158
    ETV4     0.037361 1.688133e-05           0.028158
   EPB42    -0.020447 2.157070e-05           0.031168
   CENPM    -0.051850 3.061145e-05           0.040829
 PLEKHN1     0.044331 3.372218

/home/yxcheng/AI4Science_gold_standard/stage1/Yuxuan/../utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
